### Text Extraction code for a single textbook page (a png)

In [ ]:

import surya
import argparse
import json
from collections import defaultdict

from surya.input.langs import replace_lang_with_code, get_unique_langs
from surya.input.load import load_from_folder, load_from_file, load_lang_file
from surya.model.detection.segformer import load_model as load_detection_model, load_processor as load_detection_processor
from surya.model.recognition.model import load_model as load_recognition_model
from surya.model.recognition.processor import load_processor as load_recognition_processor
from surya.model.recognition.tokenizer import _tokenize
from surya.ocr import run_ocr
from surya.postprocessing.text import draw_text_on_image
from surya.settings import settings
import os

def main(input_path,
    results_dir,
    lang_file=None,
    langs='hi,en',
    max=None,
    start_page=None,
    images=False):

    if os.path.isdir(input_path):
        images, names = load_from_folder(input_path, max, start_page)
        folder_name = os.path.basename(input_path)
    else:
        images, names = load_from_file(input_path, max, start_page)
        folder_name = os.path.basename(input_path).split(".")[0]

    if lang_file:
        # We got all of our language settings from a file
        langs = load_lang_file(lang_file, names) 
        for lang in langs:
            replace_lang_with_code(lang)
        image_langs = langs
    else:
        # We got our language settings from the input
        langs = langs.split(",")
        replace_lang_with_code(langs)
        image_langs = [langs] * len(images)

    det_processor = load_detection_processor()
    det_model = load_detection_model()

    _, lang_tokens = _tokenize("", get_unique_langs(image_langs))
    rec_model = load_recognition_model(langs=lang_tokens) # Prune model moe layer to only include languages we need
    rec_processor = load_recognition_processor()

    result_path = os.path.join(results_dir, folder_name)
    os.makedirs(result_path, exist_ok=True)

    predictions_by_image = run_ocr(images, image_langs, det_model, det_processor, rec_model, rec_processor)

    if images:
        for idx, (name, image, pred, langs) in enumerate(zip(names, images, predictions_by_image, image_langs)):
            bboxes = [l.bbox for l in pred.text_lines]
            pred_text = [l.text for l in pred.text_lines]
            page_image = draw_text_on_image(bboxes, pred_text, image.size, langs, has_math="_math" in langs)
            page_image.save(f'./temp{idx}.jpg')
            page_image.save(os.path.join(result_path, f"{name}_{idx}_text.png"))

        out_preds = defaultdict(list)
        for name, pred, image in zip(names, predictions_by_image, images):
            out_pred = pred.model_dump()
            out_pred["page"] = len(out_preds[name]) + 1
            out_preds[name].append(out_pred)

        with open(os.path.join(result_path, "results.json"), "w+", encoding="utf-8") as f:
            json.dump(out_preds, f, ensure_ascii=False)

        print(f"Wrote results to {result_path}")

        # from PIL import Image, ImageDraw
        # indices = []
        # prev = -1
        # for idx in range(0, len(out_preds['img6'][0]['text_lines'])):
        #     x_axis = out_preds['img6'][0]['text_lines'][idx]['polygon'][0][0]
            
        #     if prev != -1:
        #         if x_axis > (prev + 5):
        #             indices.append(idx)
            
        #     prev = x_axis
            
        # str = ''
        # for idx in range(0, len(out_preds['img6'][0]['text_lines'])):
        #     if idx not in indices:
        #         str = str + out_preds['img6'][0]['text_lines'][idx]['text'] + ' '
        # print(str)
        


if __name__ == "__main__":
    input_path = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Books PNGS/FED2/page_4.png'
    results_dir = '/Users/fajrzafar/Documents/Semester 8 /FYP 1'
    main(input_path, results_dir)

### Text Extraction code for Grade 1 federal book (all pages)

In [1]:
import argparse
import json
from collections import defaultdict

from surya.input.langs import replace_lang_with_code, get_unique_langs
from surya.input.load import load_from_folder, load_from_file, load_lang_file
from surya.model.detection.segformer import load_model as load_detection_model, load_processor as load_detection_processor
from surya.model.recognition.model import load_model as load_recognition_model
from surya.model.recognition.processor import load_processor as load_recognition_processor
from surya.model.recognition.tokenizer import _tokenize
from surya.ocr import run_ocr
from surya.postprocessing.text import draw_text_on_image
from surya.settings import settings
import os
import csv 



def process_folder(input_folder, results_dir):
    # Get a list of all PNG files in the folder
    png_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

    for png_file in png_files:
        # Construct the full path to the PNG file
        png_path = os.path.join(input_folder, png_file)

        # Call the main function for each PNG file
        main(png_path, results_dir)


def main(input_path,
    results_dir,
    lang_file=None,
    langs='hi,en',
    max=None,
    start_page=None,
    images=False):

    if os.path.isdir(input_path):
        images, names = load_from_folder(input_path, max, start_page)
        folder_name = os.path.basename(input_path)
    else:
        images, names = load_from_file(input_path, max, start_page)
        folder_name = os.path.basename(input_path).split(".")[0]

    if lang_file:
        # We got all of our language settings from a file
        langs = load_lang_file(lang_file, names) 
        for lang in langs:
            replace_lang_with_code(lang)
        image_langs = langs
    else:
        # We got our language settings from the input
        langs = langs.split(",")
        replace_lang_with_code(langs)
        image_langs = [langs] * len(images)

    det_processor = load_detection_processor()
    det_model = load_detection_model()

    _, lang_tokens = _tokenize("", get_unique_langs(image_langs))
    rec_model = load_recognition_model(langs=lang_tokens) # Prune model moe layer to only include languages we need
    rec_processor = load_recognition_processor()

    result_path = os.path.join(results_dir, folder_name)
    os.makedirs(result_path, exist_ok=True)

    predictions_by_image = run_ocr(images, image_langs, det_model, det_processor, rec_model, rec_processor)

    if images:
        for idx, (name, image, pred, langs) in enumerate(zip(names, images, predictions_by_image, image_langs)):
            bboxes = [l.bbox for l in pred.text_lines]
            pred_text = [l.text for l in pred.text_lines]
            page_image = draw_text_on_image(bboxes, pred_text, image.size, langs, has_math="_math" in langs)
            page_image.save(f'./temp{idx}.jpg')
            page_image.save(os.path.join(result_path, f"{name}_{idx}_text.png"))

             # Save text to CSV file
            csv_file_path = os.path.join(result_path, f"{name}_{idx}_text.csv")
            with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow(['Text'])
                for line in pred_text:
                    writer.writerow([line])


        out_preds = defaultdict(list)
        for name, pred, image in zip(names, predictions_by_image, images):
            out_pred = pred.model_dump()
            out_pred["page"] = len(out_preds[name]) + 1
            out_preds[name].append(out_pred)

        with open(os.path.join(result_path, "results.json"), "w+", encoding="utf-8") as f:
            json.dump(out_preds, f, ensure_ascii=False)

        print(f"Wrote results to {result_path}")

        # from PIL import Image, ImageDraw
        # indices = []
        # prev = -1
        # for idx in range(len(preds['text_lines'])):
        #     x_axis = preds['text_lines'][idx]['polygon'][0][0]
        #     if prev != -1 and x_axis > (prev + 5):
        #         indices.append(idx)
        #     prev = x_axis
            
        # text = ''
        # for idx in range(len(preds['text_lines'])):
        #     if idx not in indices:
        #         text += preds['text_lines'][idx]['text'] + ' '

        # print(f"Text for page {page_idx + 1}: {text}")
        


if __name__ == "__main__":
     input_folder = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Books PNGS/FED1'
     results_dir = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED1'
     process_folder(input_folder, results_dir)

Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text:   0%|                                   | 0/1 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Recognizing Text: 100%|███████████████████████████| 1/1 [00:16<00:00, 16.05s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED1/page_2
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [00:25<00:00, 25.23s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED1/page_3
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


...

Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED1/page_14
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [00:54<00:00, 54.29s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED1/page_8
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|██████████████████████████| 1/1 [02:07<00:00, 127.54s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED1/page_9


### Text Extraction code for Grade 2 federal book (all pages)

In [2]:
import argparse
import json
from collections import defaultdict

from surya.input.langs import replace_lang_with_code, get_unique_langs
from surya.input.load import load_from_folder, load_from_file, load_lang_file
from surya.model.detection.segformer import load_model as load_detection_model, load_processor as load_detection_processor
from surya.model.recognition.model import load_model as load_recognition_model
from surya.model.recognition.processor import load_processor as load_recognition_processor
from surya.model.recognition.tokenizer import _tokenize
from surya.ocr import run_ocr
from surya.postprocessing.text import draw_text_on_image
from surya.settings import settings
import os
import csv 



def process_folder(input_folder, results_dir):
    # Get a list of all PNG files in the folder
    png_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

    for png_file in png_files:
        # Construct the full path to the PNG file
        png_path = os.path.join(input_folder, png_file)

        # Call the main function for each PNG file
        main(png_path, results_dir)


def main(input_path,
    results_dir,
    lang_file=None,
    langs='hi,en',
    max=None,
    start_page=None,
    images=False):

    if os.path.isdir(input_path):
        images, names = load_from_folder(input_path, max, start_page)
        folder_name = os.path.basename(input_path)
    else:
        images, names = load_from_file(input_path, max, start_page)
        folder_name = os.path.basename(input_path).split(".")[0]

    if lang_file:
        # We got all of our language settings from a file
        langs = load_lang_file(lang_file, names) 
        for lang in langs:
            replace_lang_with_code(lang)
        image_langs = langs
    else:
        # We got our language settings from the input
        langs = langs.split(",")
        replace_lang_with_code(langs)
        image_langs = [langs] * len(images)

    det_processor = load_detection_processor()
    det_model = load_detection_model()

    _, lang_tokens = _tokenize("", get_unique_langs(image_langs))
    rec_model = load_recognition_model(langs=lang_tokens) # Prune model moe layer to only include languages we need
    rec_processor = load_recognition_processor()

    result_path = os.path.join(results_dir, folder_name)
    os.makedirs(result_path, exist_ok=True)

    predictions_by_image = run_ocr(images, image_langs, det_model, det_processor, rec_model, rec_processor)

    if images:
        for idx, (name, image, pred, langs) in enumerate(zip(names, images, predictions_by_image, image_langs)):
            bboxes = [l.bbox for l in pred.text_lines]
            pred_text = [l.text for l in pred.text_lines]
            page_image = draw_text_on_image(bboxes, pred_text, image.size, langs, has_math="_math" in langs)
            page_image.save(f'./temp{idx}.jpg')
            page_image.save(os.path.join(result_path, f"{name}_{idx}_text.png"))

             # Save text to CSV file
            csv_file_path = os.path.join(result_path, f"{name}_{idx}_text.csv")
            with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow(['Text'])
                for line in pred_text:
                    writer.writerow([line])


        out_preds = defaultdict(list)
        for name, pred, image in zip(names, predictions_by_image, images):
            out_pred = pred.model_dump()
            out_pred["page"] = len(out_preds[name]) + 1
            out_preds[name].append(out_pred)

        with open(os.path.join(result_path, "results.json"), "w+", encoding="utf-8") as f:
            json.dump(out_preds, f, ensure_ascii=False)

        print(f"Wrote results to {result_path}")

        # from PIL import Image, ImageDraw
        # indices = []
        # prev = -1
        # for idx in range(len(preds['text_lines'])):
        #     x_axis = preds['text_lines'][idx]['polygon'][0][0]
        #     if prev != -1 and x_axis > (prev + 5):
        #         indices.append(idx)
        #     prev = x_axis
            
        # text = ''
        # for idx in range(len(preds['text_lines'])):
        #     if idx not in indices:
        #         text += preds['text_lines'][idx]['text'] + ' '

        # print(f"Text for page {page_idx + 1}: {text}")
        


if __name__ == "__main__":
     input_folder = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Books PNGS/FED2'
     results_dir = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED2'
     process_folder(input_folder, results_dir)

Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [00:33<00:00, 33.62s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED2/page_2
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [00:43<00:00, 43.33s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED2/page_3
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


...

Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED2/page_12
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [00:44<00:00, 44.55s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED2/page_8
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [00:24<00:00, 24.35s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED2/page_9


### Text Extraction code for Grade 3 federal book (all pages)

In [3]:
import argparse
import json
from collections import defaultdict

from surya.input.langs import replace_lang_with_code, get_unique_langs
from surya.input.load import load_from_folder, load_from_file, load_lang_file
from surya.model.detection.segformer import load_model as load_detection_model, load_processor as load_detection_processor
from surya.model.recognition.model import load_model as load_recognition_model
from surya.model.recognition.processor import load_processor as load_recognition_processor
from surya.model.recognition.tokenizer import _tokenize
from surya.ocr import run_ocr
from surya.postprocessing.text import draw_text_on_image
from surya.settings import settings
import os
import csv 



def process_folder(input_folder, results_dir):
    # Get a list of all PNG files in the folder
    png_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

    for png_file in png_files:
        # Construct the full path to the PNG file
        png_path = os.path.join(input_folder, png_file)

        # Call the main function for each PNG file
        main(png_path, results_dir)


def main(input_path,
    results_dir,
    lang_file=None,
    langs='hi,en',
    max=None,
    start_page=None,
    images=False):

    if os.path.isdir(input_path):
        images, names = load_from_folder(input_path, max, start_page)
        folder_name = os.path.basename(input_path)
    else:
        images, names = load_from_file(input_path, max, start_page)
        folder_name = os.path.basename(input_path).split(".")[0]

    if lang_file:
        # We got all of our language settings from a file
        langs = load_lang_file(lang_file, names) 
        for lang in langs:
            replace_lang_with_code(lang)
        image_langs = langs
    else:
        # We got our language settings from the input
        langs = langs.split(",")
        replace_lang_with_code(langs)
        image_langs = [langs] * len(images)

    det_processor = load_detection_processor()
    det_model = load_detection_model()

    _, lang_tokens = _tokenize("", get_unique_langs(image_langs))
    rec_model = load_recognition_model(langs=lang_tokens) # Prune model moe layer to only include languages we need
    rec_processor = load_recognition_processor()

    result_path = os.path.join(results_dir, folder_name)
    os.makedirs(result_path, exist_ok=True)

    predictions_by_image = run_ocr(images, image_langs, det_model, det_processor, rec_model, rec_processor)

    if images:
        for idx, (name, image, pred, langs) in enumerate(zip(names, images, predictions_by_image, image_langs)):
            bboxes = [l.bbox for l in pred.text_lines]
            pred_text = [l.text for l in pred.text_lines]
            page_image = draw_text_on_image(bboxes, pred_text, image.size, langs, has_math="_math" in langs)
            page_image.save(f'./temp{idx}.jpg')
            page_image.save(os.path.join(result_path, f"{name}_{idx}_text.png"))

             # Save text to CSV file
            csv_file_path = os.path.join(result_path, f"{name}_{idx}_text.csv")
            with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow(['Text'])
                for line in pred_text:
                    writer.writerow([line])


        out_preds = defaultdict(list)
        for name, pred, image in zip(names, predictions_by_image, images):
            out_pred = pred.model_dump()
            out_pred["page"] = len(out_preds[name]) + 1
            out_preds[name].append(out_pred)

        with open(os.path.join(result_path, "results.json"), "w+", encoding="utf-8") as f:
            json.dump(out_preds, f, ensure_ascii=False)

        print(f"Wrote results to {result_path}")

        # from PIL import Image, ImageDraw
        # indices = []
        # prev = -1
        # for idx in range(len(preds['text_lines'])):
        #     x_axis = preds['text_lines'][idx]['polygon'][0][0]
        #     if prev != -1 and x_axis > (prev + 5):
        #         indices.append(idx)
        #     prev = x_axis
            
        # text = ''
        # for idx in range(len(preds['text_lines'])):
        #     if idx not in indices:
        #         text += preds['text_lines'][idx]['text'] + ' '

        # print(f"Text for page {page_idx + 1}: {text}")
        


if __name__ == "__main__":
     input_folder = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Books PNGS/FED3'
     results_dir = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED3'
     process_folder(input_folder, results_dir)

Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|██████████████████████████| 1/1 [03:46<00:00, 226.78s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED3/page_2
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [01:17<00:00, 77.31s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED3/page_3
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


...

Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED3/page_12
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|██████████████████████████| 1/1 [09:53<00:00, 593.01s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED3/page_8
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [01:07<00:00, 67.18s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/FED3/page_9


### Text Extraction code for Grade1 PTB book (all pages)

In [4]:
import argparse
import json
from collections import defaultdict

from surya.input.langs import replace_lang_with_code, get_unique_langs
from surya.input.load import load_from_folder, load_from_file, load_lang_file
from surya.model.detection.segformer import load_model as load_detection_model, load_processor as load_detection_processor
from surya.model.recognition.model import load_model as load_recognition_model
from surya.model.recognition.processor import load_processor as load_recognition_processor
from surya.model.recognition.tokenizer import _tokenize
from surya.ocr import run_ocr
from surya.postprocessing.text import draw_text_on_image
from surya.settings import settings
import os
import csv 



def process_folder(input_folder, results_dir):
    # Get a list of all PNG files in the folder
    png_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

    for png_file in png_files:
        # Construct the full path to the PNG file
        png_path = os.path.join(input_folder, png_file)

        # Call the main function for each PNG file
        main(png_path, results_dir)


def main(input_path,
    results_dir,
    lang_file=None,
    langs='hi,en',
    max=None,
    start_page=None,
    images=False):

    if os.path.isdir(input_path):
        images, names = load_from_folder(input_path, max, start_page)
        folder_name = os.path.basename(input_path)
    else:
        images, names = load_from_file(input_path, max, start_page)
        folder_name = os.path.basename(input_path).split(".")[0]

    if lang_file:
        # We got all of our language settings from a file
        langs = load_lang_file(lang_file, names) 
        for lang in langs:
            replace_lang_with_code(lang)
        image_langs = langs
    else:
        # We got our language settings from the input
        langs = langs.split(",")
        replace_lang_with_code(langs)
        image_langs = [langs] * len(images)

    det_processor = load_detection_processor()
    det_model = load_detection_model()

    _, lang_tokens = _tokenize("", get_unique_langs(image_langs))
    rec_model = load_recognition_model(langs=lang_tokens) # Prune model moe layer to only include languages we need
    rec_processor = load_recognition_processor()

    result_path = os.path.join(results_dir, folder_name)
    os.makedirs(result_path, exist_ok=True)

    predictions_by_image = run_ocr(images, image_langs, det_model, det_processor, rec_model, rec_processor)

    if images:
        for idx, (name, image, pred, langs) in enumerate(zip(names, images, predictions_by_image, image_langs)):
            bboxes = [l.bbox for l in pred.text_lines]
            pred_text = [l.text for l in pred.text_lines]
            page_image = draw_text_on_image(bboxes, pred_text, image.size, langs, has_math="_math" in langs)
            page_image.save(f'./temp{idx}.jpg')
            page_image.save(os.path.join(result_path, f"{name}_{idx}_text.png"))

             # Save text to CSV file
            csv_file_path = os.path.join(result_path, f"{name}_{idx}_text.csv")
            with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow(['Text'])
                for line in pred_text:
                    writer.writerow([line])


        out_preds = defaultdict(list)
        for name, pred, image in zip(names, predictions_by_image, images):
            out_pred = pred.model_dump()
            out_pred["page"] = len(out_preds[name]) + 1
            out_preds[name].append(out_pred)

        with open(os.path.join(result_path, "results.json"), "w+", encoding="utf-8") as f:
            json.dump(out_preds, f, ensure_ascii=False)

        print(f"Wrote results to {result_path}")

        # from PIL import Image, ImageDraw
        # indices = []
        # prev = -1
        # for idx in range(len(preds['text_lines'])):
        #     x_axis = preds['text_lines'][idx]['polygon'][0][0]
        #     if prev != -1 and x_axis > (prev + 5):
        #         indices.append(idx)
        #     prev = x_axis
            
        # text = ''
        # for idx in range(len(preds['text_lines'])):
        #     if idx not in indices:
        #         text += preds['text_lines'][idx]['text'] + ' '

        # print(f"Text for page {page_idx + 1}: {text}")
        


if __name__ == "__main__":
     input_folder = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Books PNGS/PTB1'
     results_dir = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB1'
     process_folder(input_folder, results_dir)

Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [01:38<00:00, 98.61s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB1/page_2
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [01:28<00:00, 88.93s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB1/page_3
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


...

Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB1/page_14
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|██████████████████████████| 1/1 [03:14<00:00, 194.15s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB1/page_8
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [00:47<00:00, 47.48s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB1/page_9


In [2]:
import argparse
import json
from collections import defaultdict

from surya.input.langs import replace_lang_with_code, get_unique_langs
from surya.input.load import load_from_folder, load_from_file, load_lang_file
from surya.model.detection.segformer import load_model as load_detection_model, load_processor as load_detection_processor
from surya.model.recognition.model import load_model as load_recognition_model
from surya.model.recognition.processor import load_processor as load_recognition_processor
from surya.model.recognition.tokenizer import _tokenize
from surya.ocr import run_ocr
from surya.postprocessing.text import draw_text_on_image
from surya.settings import settings
import os
import csv 



def process_folder(input_folder, results_dir):
    # Get a list of all PNG files in the folder
    png_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

    for png_file in png_files:
        # Construct the full path to the PNG file
        png_path = os.path.join(input_folder, png_file)

        # Call the main function for each PNG file
        main(png_path, results_dir)


def main(input_path,
    results_dir,
    lang_file=None,
    langs='hi,en',
    max=None,
    start_page=None,
    images=False):

    if os.path.isdir(input_path):
        images, names = load_from_folder(input_path, max, start_page)
        folder_name = os.path.basename(input_path)
    else:
        images, names = load_from_file(input_path, max, start_page)
        folder_name = os.path.basename(input_path).split(".")[0]

    if lang_file:
        # We got all of our language settings from a file
        langs = load_lang_file(lang_file, names) 
        for lang in langs:
            replace_lang_with_code(lang)
        image_langs = langs
    else:
        # We got our language settings from the input
        langs = langs.split(",")
        replace_lang_with_code(langs)
        image_langs = [langs] * len(images)

    det_processor = load_detection_processor()
    det_model = load_detection_model()

    _, lang_tokens = _tokenize("", get_unique_langs(image_langs))
    rec_model = load_recognition_model(langs=lang_tokens) # Prune model moe layer to only include languages we need
    rec_processor = load_recognition_processor()

    result_path = os.path.join(results_dir, folder_name)
    os.makedirs(result_path, exist_ok=True)

    predictions_by_image = run_ocr(images, image_langs, det_model, det_processor, rec_model, rec_processor)

    if images:
        for idx, (name, image, pred, langs) in enumerate(zip(names, images, predictions_by_image, image_langs)):
            bboxes = [l.bbox for l in pred.text_lines]
            pred_text = [l.text for l in pred.text_lines]
            page_image = draw_text_on_image(bboxes, pred_text, image.size, langs, has_math="_math" in langs)
            page_image.save(f'./temp{idx}.jpg')
            page_image.save(os.path.join(result_path, f"{name}_{idx}_text.png"))

             # Save text to CSV file
            csv_file_path = os.path.join(result_path, f"{name}_{idx}_text.csv")
            with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow(['Text'])
                for line in pred_text:
                    writer.writerow([line])


        out_preds = defaultdict(list)
        for name, pred, image in zip(names, predictions_by_image, images):
            out_pred = pred.model_dump()
            out_pred["page"] = len(out_preds[name]) + 1
            out_preds[name].append(out_pred)

        with open(os.path.join(result_path, "results.json"), "w+", encoding="utf-8") as f:
            json.dump(out_preds, f, ensure_ascii=False)

        print(f"Wrote results to {result_path}")

        # from PIL import Image, ImageDraw
        # indices = []
        # prev = -1
        # for idx in range(len(preds['text_lines'])):
        #     x_axis = preds['text_lines'][idx]['polygon'][0][0]
        #     if prev != -1 and x_axis > (prev + 5):
        #         indices.append(idx)
        #     prev = x_axis
            
        # text = ''
        # for idx in range(len(preds['text_lines'])):
        #     if idx not in indices:
        #         text += preds['text_lines'][idx]['text'] + ' '

        # print(f"Text for page {page_idx + 1}: {text}")
        


if __name__ == "__main__":
     input_folder = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Books PNGS/PTB2'
     results_dir = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB2'
     process_folder(input_folder, results_dir)

Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text:   0%|                                   | 0/1 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Recognizing Text: 100%|███████████████████████████| 1/1 [00:13<00:00, 13.07s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB2/page_2
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [00:29<00:00, 29.88s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB2/page_3
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


...

Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB2/page_14
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [00:19<00:00, 19.05s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB2/page_8
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|██████████████████████████| 1/1 [12:44<00:00, 764.34s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB2/page_9


In [1]:
import argparse
import json
from collections import defaultdict

from surya.input.langs import replace_lang_with_code, get_unique_langs
from surya.input.load import load_from_folder, load_from_file, load_lang_file
from surya.model.detection.segformer import load_model as load_detection_model, load_processor as load_detection_processor
from surya.model.recognition.model import load_model as load_recognition_model
from surya.model.recognition.processor import load_processor as load_recognition_processor
from surya.model.recognition.tokenizer import _tokenize
from surya.ocr import run_ocr
from surya.postprocessing.text import draw_text_on_image
from surya.settings import settings
import os
import csv 



def process_folder(input_folder, results_dir):
    # Get a list of all PNG files in the folder
    png_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

    for png_file in png_files:
        # Construct the full path to the PNG file
        png_path = os.path.join(input_folder, png_file)

        # Call the main function for each PNG file
        main(png_path, results_dir)


def main(input_path,
    results_dir,
    lang_file=None,
    langs='hi,en',
    max=None,
    start_page=None,
    images=False):

    if os.path.isdir(input_path):
        images, names = load_from_folder(input_path, max, start_page)
        folder_name = os.path.basename(input_path)
    else:
        images, names = load_from_file(input_path, max, start_page)
        folder_name = os.path.basename(input_path).split(".")[0]

    if lang_file:
        # We got all of our language settings from a file
        langs = load_lang_file(lang_file, names) 
        for lang in langs:
            replace_lang_with_code(lang)
        image_langs = langs
    else:
        # We got our language settings from the input
        langs = langs.split(",")
        replace_lang_with_code(langs)
        image_langs = [langs] * len(images)

    det_processor = load_detection_processor()
    det_model = load_detection_model()

    _, lang_tokens = _tokenize("", get_unique_langs(image_langs))
    rec_model = load_recognition_model(langs=lang_tokens) # Prune model moe layer to only include languages we need
    rec_processor = load_recognition_processor()

    result_path = os.path.join(results_dir, folder_name)
    os.makedirs(result_path, exist_ok=True)

    predictions_by_image = run_ocr(images, image_langs, det_model, det_processor, rec_model, rec_processor)

    if images:
        for idx, (name, image, pred, langs) in enumerate(zip(names, images, predictions_by_image, image_langs)):
            bboxes = [l.bbox for l in pred.text_lines]
            pred_text = [l.text for l in pred.text_lines]
            page_image = draw_text_on_image(bboxes, pred_text, image.size, langs, has_math="_math" in langs)
            page_image.save(f'./temp{idx}.jpg')
            page_image.save(os.path.join(result_path, f"{name}_{idx}_text.png"))

             # Save text to CSV file
            csv_file_path = os.path.join(result_path, f"{name}_{idx}_text.csv")
            with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow(['Text'])
                for line in pred_text:
                    writer.writerow([line])


        out_preds = defaultdict(list)
        for name, pred, image in zip(names, predictions_by_image, images):
            out_pred = pred.model_dump()
            out_pred["page"] = len(out_preds[name]) + 1
            out_preds[name].append(out_pred)

        with open(os.path.join(result_path, "results.json"), "w+", encoding="utf-8") as f:
            json.dump(out_preds, f, ensure_ascii=False)

        print(f"Wrote results to {result_path}")

        # from PIL import Image, ImageDraw
        # indices = []
        # prev = -1
        # for idx in range(len(preds['text_lines'])):
        #     x_axis = preds['text_lines'][idx]['polygon'][0][0]
        #     if prev != -1 and x_axis > (prev + 5):
        #         indices.append(idx)
        #     prev = x_axis
            
        # text = ''
        # for idx in range(len(preds['text_lines'])):
        #     if idx not in indices:
        #         text += preds['text_lines'][idx]['text'] + ' '

        # print(f"Text for page {page_idx + 1}: {text}")
        


if __name__ == "__main__":
     input_folder = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Books PNGS/PTB3'
     results_dir = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB3'
     process_folder(input_folder, results_dir)

Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text:   0%|                                   | 0/1 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Recognizing Text: 100%|███████████████████████████| 1/1 [00:14<00:00, 14.73s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB3/page_2
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [00:38<00:00, 38.67s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB3/page_3
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


...

Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB3/page_31
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|██████████████████████████| 1/1 [01:43<00:00, 103.65s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/PTB3/page_30
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text:   0%|                                   | 0/1 [00:09<?, ?it/s]


RuntimeError: MPS backend out of memory (MPS allocated: 7.18 GB, other allocations: 1.58 GB, max allowed: 9.07 GB). Tried to allocate 332.28 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [2]:
import argparse
import json
from collections import defaultdict

from surya.input.langs import replace_lang_with_code, get_unique_langs
from surya.input.load import load_from_folder, load_from_file, load_lang_file
from surya.model.detection.segformer import load_model as load_detection_model, load_processor as load_detection_processor
from surya.model.recognition.model import load_model as load_recognition_model
from surya.model.recognition.processor import load_processor as load_recognition_processor
from surya.model.recognition.tokenizer import _tokenize
from surya.ocr import run_ocr
from surya.postprocessing.text import draw_text_on_image
from surya.settings import settings
import os
import csv 



def process_folder(input_folder, results_dir):
    # Get a list of all PNG files in the folder
    png_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

    for png_file in png_files:
        # Construct the full path to the PNG file
        png_path = os.path.join(input_folder, png_file)

        # Call the main function for each PNG file
        main(png_path, results_dir)


def main(input_path,
    results_dir,
    lang_file=None,
    langs='hi,en',
    max=None,
    start_page=None,
    images=False):

    if os.path.isdir(input_path):
        images, names = load_from_folder(input_path, max, start_page)
        folder_name = os.path.basename(input_path)
    else:
        images, names = load_from_file(input_path, max, start_page)
        folder_name = os.path.basename(input_path).split(".")[0]

    if lang_file:
        # We got all of our language settings from a file
        langs = load_lang_file(lang_file, names) 
        for lang in langs:
            replace_lang_with_code(lang)
        image_langs = langs
    else:
        # We got our language settings from the input
        langs = langs.split(",")
        replace_lang_with_code(langs)
        image_langs = [langs] * len(images)

    det_processor = load_detection_processor()
    det_model = load_detection_model()

    _, lang_tokens = _tokenize("", get_unique_langs(image_langs))
    rec_model = load_recognition_model(langs=lang_tokens) # Prune model moe layer to only include languages we need
    rec_processor = load_recognition_processor()

    result_path = os.path.join(results_dir, folder_name)
    os.makedirs(result_path, exist_ok=True)

    predictions_by_image = run_ocr(images, image_langs, det_model, det_processor, rec_model, rec_processor)

    if images:
        for idx, (name, image, pred, langs) in enumerate(zip(names, images, predictions_by_image, image_langs)):
            bboxes = [l.bbox for l in pred.text_lines]
            pred_text = [l.text for l in pred.text_lines]
            page_image = draw_text_on_image(bboxes, pred_text, image.size, langs, has_math="_math" in langs)
            page_image.save(f'./temp{idx}.jpg')
            page_image.save(os.path.join(result_path, f"{name}_{idx}_text.png"))

             # Save text to CSV file
            csv_file_path = os.path.join(result_path, f"{name}_{idx}_text.csv")
            with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow(['Text'])
                for line in pred_text:
                    writer.writerow([line])


        out_preds = defaultdict(list)
        for name, pred, image in zip(names, predictions_by_image, images):
            out_pred = pred.model_dump()
            out_pred["page"] = len(out_preds[name]) + 1
            out_preds[name].append(out_pred)

        with open(os.path.join(result_path, "results.json"), "w+", encoding="utf-8") as f:
            json.dump(out_preds, f, ensure_ascii=False)

        print(f"Wrote results to {result_path}")

        # from PIL import Image, ImageDraw
        # indices = []
        # prev = -1
        # for idx in range(len(preds['text_lines'])):
        #     x_axis = preds['text_lines'][idx]['polygon'][0][0]
        #     if prev != -1 and x_axis > (prev + 5):
        #         indices.append(idx)
        #     prev = x_axis
            
        # text = ''
        # for idx in range(len(preds['text_lines'])):
        #     if idx not in indices:
        #         text += preds['text_lines'][idx]['text'] + ' '

        # print(f"Text for page {page_idx + 1}: {text}")
        


if __name__ == "__main__":
     input_folder = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Books PNGS/SINDH1'
     results_dir = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/SINDH1'
     process_folder(input_folder, results_dir)

Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [01:15<00:00, 75.00s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/SINDH1/page_2
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [00:58<00:00, 58.96s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/SINDH1/page_3
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


...

Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/SINDH1/page_1
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [01:34<00:00, 94.44s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/SINDH1/page_4
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text:   0%|                                   | 0/1 [00:08<?, ?it/s]


RuntimeError: MPS backend out of memory (MPS allocated: 7.22 GB, other allocations: 1.58 GB, max allowed: 9.07 GB). Tried to allocate 332.28 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [3]:
import argparse
import json
from collections import defaultdict

from surya.input.langs import replace_lang_with_code, get_unique_langs
from surya.input.load import load_from_folder, load_from_file, load_lang_file
from surya.model.detection.segformer import load_model as load_detection_model, load_processor as load_detection_processor
from surya.model.recognition.model import load_model as load_recognition_model
from surya.model.recognition.processor import load_processor as load_recognition_processor
from surya.model.recognition.tokenizer import _tokenize
from surya.ocr import run_ocr
from surya.postprocessing.text import draw_text_on_image
from surya.settings import settings
import os
import csv 



def process_folder(input_folder, results_dir):
    # Get a list of all PNG files in the folder
    png_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

    for png_file in png_files:
        # Construct the full path to the PNG file
        png_path = os.path.join(input_folder, png_file)

        # Call the main function for each PNG file
        main(png_path, results_dir)


def main(input_path,
    results_dir,
    lang_file=None,
    langs='hi,en',
    max=None,
    start_page=None,
    images=False):

    if os.path.isdir(input_path):
        images, names = load_from_folder(input_path, max, start_page)
        folder_name = os.path.basename(input_path)
    else:
        images, names = load_from_file(input_path, max, start_page)
        folder_name = os.path.basename(input_path).split(".")[0]

    if lang_file:
        # We got all of our language settings from a file
        langs = load_lang_file(lang_file, names) 
        for lang in langs:
            replace_lang_with_code(lang)
        image_langs = langs
    else:
        # We got our language settings from the input
        langs = langs.split(",")
        replace_lang_with_code(langs)
        image_langs = [langs] * len(images)

    det_processor = load_detection_processor()
    det_model = load_detection_model()

    _, lang_tokens = _tokenize("", get_unique_langs(image_langs))
    rec_model = load_recognition_model(langs=lang_tokens) # Prune model moe layer to only include languages we need
    rec_processor = load_recognition_processor()

    result_path = os.path.join(results_dir, folder_name)
    os.makedirs(result_path, exist_ok=True)

    predictions_by_image = run_ocr(images, image_langs, det_model, det_processor, rec_model, rec_processor)

    if images:
        for idx, (name, image, pred, langs) in enumerate(zip(names, images, predictions_by_image, image_langs)):
            bboxes = [l.bbox for l in pred.text_lines]
            pred_text = [l.text for l in pred.text_lines]
            page_image = draw_text_on_image(bboxes, pred_text, image.size, langs, has_math="_math" in langs)
            page_image.save(f'./temp{idx}.jpg')
            page_image.save(os.path.join(result_path, f"{name}_{idx}_text.png"))

             # Save text to CSV file
            csv_file_path = os.path.join(result_path, f"{name}_{idx}_text.csv")
            with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow(['Text'])
                for line in pred_text:
                    writer.writerow([line])


        out_preds = defaultdict(list)
        for name, pred, image in zip(names, predictions_by_image, images):
            out_pred = pred.model_dump()
            out_pred["page"] = len(out_preds[name]) + 1
            out_preds[name].append(out_pred)

        with open(os.path.join(result_path, "results.json"), "w+", encoding="utf-8") as f:
            json.dump(out_preds, f, ensure_ascii=False)

        print(f"Wrote results to {result_path}")

        # from PIL import Image, ImageDraw
        # indices = []
        # prev = -1
        # for idx in range(len(preds['text_lines'])):
        #     x_axis = preds['text_lines'][idx]['polygon'][0][0]
        #     if prev != -1 and x_axis > (prev + 5):
        #         indices.append(idx)
        #     prev = x_axis
            
        # text = ''
        # for idx in range(len(preds['text_lines'])):
        #     if idx not in indices:
        #         text += preds['text_lines'][idx]['text'] + ' '

        # print(f"Text for page {page_idx + 1}: {text}")
        


if __name__ == "__main__":
     input_folder = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Books PNGS/SINDH2'
     results_dir = '/Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/SINDH2'
     process_folder(input_folder, results_dir)

Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [00:32<00:00, 32.21s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/SINDH2/page_2
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [01:10<00:00, 70.22s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/SINDH2/page_3
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


...

Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/SINDH2/page_20
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text: 100%|███████████████████████████| 1/1 [00:23<00:00, 23.31s/it]


Wrote results to /Users/fajrzafar/Documents/Semester 8 /FYP 1/Extracted_Text/SINDH2/page_10
Loading detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loading recognition model vikp/surya_rec on device mps with dtype torch.float16


Recognizing Text:   0%|                                   | 0/1 [00:12<?, ?it/s]


RuntimeError: MPS backend out of memory (MPS allocated: 7.27 GB, other allocations: 1.76 GB, max allowed: 9.07 GB). Tried to allocate 58.19 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).